In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
#Загрузчик с диска гугл
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/Colab Notebooks/NLP/9

/content/drive/MyDrive/Colab Notebooks/NLP/9


In [4]:
!ls *.txt

hobbit_ili_tuda_i_obratno_per_m_kamenkovih_s_stepa.txt


In [5]:
path_to_file = 'hobbit_ili_tuda_i_obratno_per_m_kamenkovih_s_stepa.txt'

In [6]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 636492 characters


In [7]:
print(text[:500])

6lib.ru - Электронная Библиотека
Название книги: Хоббит, или Туда и обратно (пер. М. Каменкович, С. Степанов)
Автор(ы): Толкиен Джон
Жанр: Фэнтези
Адрес книги: http://www.6lib.ru/books/hobbit_ili_tuda_i_obratno_per_m_kamenkovih_s_stepanov_-11249.html

– Электронная Библиотека
Название книги: Хоббит, или Туда и обратно (пер. М. Каменкович, С. Степанов)
Автор(ы): Толкиен Джон Роналд Руэл
Жанр: Фэнтези
Адрес книги: /http://www.6lib.ru/11249-hobbit_ili_tuda_i_obratno_per_m_kamenkovih_s_stepanov_


In [8]:
text = text + text

In [9]:
# Количество уникальных символов в тексте
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

151 unique characters


In [10]:
# Индексация уникальных символов 
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [11]:
text_as_int, text, len(text_as_int), len(text)

(array([ 20,  64,  61, ..., 129, 121, 123]),
 '6lib.ru - Электронная Библиотека\r\nНазвание книги: Хоббит, или Туда и обратно (пер. М. Каменкович, С. Степанов)\r\nАвтор(ы): Толкиен Джон\r\nЖанр: Фэнтези\r\nАдрес книги: http://www.6lib.ru/books/hobbit_ili_tuda_i_obratno_per_m_kamenkovih_s_stepanov_-11249.html\n\n–\xa0Электронная Библиотека\nНазвание книги: Хоббит, или Туда и обратно (пер. М. Каменкович, С. Степанов)\nАвтор(ы): Толкиен Джон Роналд Руэл\nЖанр: Фэнтези\nАдрес книги: /http://www.6lib.ru/11249-hobbit_ili_tuda_i_obratno_per_m_kamenkovih_s_stepanov_.html\nАннотация: Повесть – сказка Дж. Р. Р. Толкина `Хоббит` является своего рода вступлением к трилогии `Властелин Колец`. Смешная и увлекательная, первоначально адресованная детям, она тем не менее вводит нас в сложный и многообразный толкиновский мир, подробно описанный на страницах трилогии. Предлагаемый читателю перевод выполнен с максимальным учетом связей между повестью и трилогией. Перевод снабжен подробными комментариями, 

In [12]:
# Максимальная длина предложения
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Обучающие примеры / таргет
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

6
l
i
b
.


In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'6lib.ru - Электронная Библиотека\r\nНазвание книги: Хоббит, или Туда и обратно (пер. М. Каменкович, С. '
'Степанов)\r\nАвтор(ы): Толкиен Джон\r\nЖанр: Фэнтези\r\nАдрес книги: http://www.6lib.ru/books/hobbit_ili_tu'
'da_i_obratno_per_m_kamenkovih_s_stepanov_-11249.html\n\n–\xa0Электронная Библиотека\nНазвание книги: Хоббит'
', или Туда и обратно (пер. М. Каменкович, С. Степанов)\nАвтор(ы): Толкиен Джон Роналд Руэл\nЖанр: Фэнте'
'зи\nАдрес книги: /http://www.6lib.ru/11249-hobbit_ili_tuda_i_obratno_per_m_kamenkovih_s_stepanov_.html'


In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [15]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '6lib.ru - Электронная Библиотека\r\nНазвание книги: Хоббит, или Туда и обратно (пер. М. Каменкович, С.'
Target data: 'lib.ru - Электронная Библиотека\r\nНазвание книги: Хоббит, или Туда и обратно (пер. М. Каменкович, С. '


In [16]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 151) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           38656     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 151)           154775    
                                                                 
Total params: 30,618,519
Trainable params: 30,618,519
No

In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'льких милях южнее Каррока. Там был довольно глубокий брод, через который вполне можно было бы перепр'

Next Char Predictions: 
 '?;«ц(IP\nчу6xIаЙБ"йz\nPzBРOPТЖJfhSзф,–wYqйIaИ…Бън_ЛхшрBA3ю c\'qГВNз5тЖdq4nв4T\'"abЧаЖсщBЦлYщи\rЯжО с»jЬхн'


In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 151)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.0173655


In [25]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [26]:
!rm -rf ./training_checkpoints

In [27]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [28]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

### Execute the training

In [29]:
EPOCHS = 20

In [30]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
196/196 [==============================] - 70s 326ms/step - loss: 3.0530
Epoch 2/20
196/196 [==============================] - 70s 345ms/step - loss: 2.3058
Epoch 3/20
196/196 [==============================] - 71s 353ms/step - loss: 1.8827
Epoch 4/20
196/196 [==============================] - 69s 346ms/step - loss: 1.6240
Epoch 5/20
196/196 [==============================] - 70s 352ms/step - loss: 1.4742
Epoch 6/20
196/196 [==============================] - 69s 347ms/step - loss: 1.3607
Epoch 7/20
196/196 [==============================] - 70s 352ms/step - loss: 1.2531
Epoch 8/20
196/196 [==============================] - 69s 347ms/step - loss: 1.1371
Epoch 9/20
196/196 [==============================] - 70s 350ms/step - loss: 1.0082
Epoch 10/20
196/196 [==============================] - 69s 347ms/step - loss: 0.8743
Epoch 11/20
196/196 [==============================] - 69s 345ms/step - loss: 0.7380
Epoch 12/20
196/196 [==============================] - 71s 358ms/step - lo

## Generate text

In [31]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_18'

In [32]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            38656     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 151)            154775    
                                                                 
Total params: 30,618,519
Trainable params: 30,618,519


In [34]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [35]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже короля, не было и Дейла. Только честно, словно чудовищная и составили эту карту. Однако хотелось бы увидеть их еще раз. Можно сказать, что прошлой ночью я начно придется утихать с гномами?
– Ну, что не завалил же! Мы бы и пикнуть не успели! По меньшей мере по шесть нечто ужасное проклятие! Он заставил я. – Ты намерен расправиться с Драконом. Но Дракон этот не только силен и страшен – он стал отвратительным сглатывающим звуком, они быстро придвинули к очагу стоявшие у стены козлы и положили на не


In [36]:
len(text_)

515